# Premier League Player Role Discovery - Phase 1: Data Cleaning

This notebook handles the data cleaning process for the Premier League Player Role Discovery project. It loads player statistics from multiple CSV files, harmonizes column names, aggregates player data across teams, filters for outfield players with sufficient minutes, and saves the cleaned dataset.


## 1. Import Libraries


In [116]:
import os
import glob
import json
import pandas as pd
import numpy as np
from typing import Dict, List, Optional, Tuple, Set
import warnings

# suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# set display options for better dataframe viewing
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)


## 2. Define Constants and Paths


In [117]:
# define paths
RAW_DATA_DIR = '../data/raw/fbref_2024_25/'
PROCESSED_DATA_DIR = '../data/processed/'
OUTPUT_FILE = os.path.join(PROCESSED_DATA_DIR, 'player_stats_cleaned.csv')
SCHEMA_FILE = os.path.join(PROCESSED_DATA_DIR, 'data_schema.json')

# minimum minutes threshold (as per requirements)
MIN_MINUTES = 600

# list of positions to exclude (goalkeepers)
EXCLUDED_POSITIONS = ['GK']

# key columns that must be present in the final dataset
REQUIRED_COLUMNS = ['player', 'team', 'position', 'minutes']


## 3. Helper Functions


In [118]:
def list_csv_files(directory: str) -> List[str]:
    """list all csv files in the given directory"""
    return glob.glob(os.path.join(directory, '*.csv'))

def clean_column_names(df: pd.DataFrame) -> pd.DataFrame:
    """standardize column names to lowercase with underscores"""
    # create a copy to avoid modifying the original dataframe
    df = df.copy()
    
    # clean column names
    df.columns = [
        col.lower()
           .replace(' ', '_')
           .replace('/', '_per_')
           .replace('%', 'pct')
           .replace('+', 'plus')
           .replace('-', '_')
           .replace('(', '')
           .replace(')', '')
           .replace('.', '')
           .replace('__', '_')
        for col in df.columns
    ]
    
    return df

def identify_player_column(df: pd.DataFrame) -> Optional[str]:
    """identify the column containing player names"""
    # For FBref data, the column is always 'Player'
    if 'Player' in df.columns:
        return 'Player'
    
    # Fallbacks for other formats
    player_column_candidates = ['player', 'player_name', 'name']
    
    # Check exact matches
    for col in player_column_candidates:
        if col in df.columns:
            return col
    
    # Case insensitive search
    for col in df.columns:
        if any(c.lower() == col.lower() for c in player_column_candidates):
            return col
    
    # If we have a row_id column (added during loading), use that as a last resort
    if 'row_id' in df.columns:
        return 'row_id'
    
    return None

def identify_minutes_column(df: pd.DataFrame) -> Optional[str]:
    """identify the column containing minutes played"""
    # For FBref data, the column is always 'Min'
    if 'Min' in df.columns:
        return 'Min'
    
    # Fallbacks for other formats
    minutes_column_candidates = ['minutes', 'min', 'minutes_played', 'mp', 'MP']
    
    # Check exact matches
    for col in minutes_column_candidates:
        if col in df.columns:
            return col
    
    # Case insensitive search
    for col in df.columns:
        if any(c.lower() == col.lower() for c in minutes_column_candidates):
            return col
            
    return None

def identify_position_column(df: pd.DataFrame) -> Optional[str]:
    """identify the column containing player positions"""
    # For FBref data, the column is always 'Pos'
    if 'Pos' in df.columns:
        return 'Pos'
    
    # Fallbacks for other formats
    position_column_candidates = ['position', 'pos', 'player_position']
    
    # Check exact matches
    for col in position_column_candidates:
        if col in df.columns:
            return col
    
    # Case insensitive search
    for col in df.columns:
        if any(c.lower() == col.lower() for c in position_column_candidates):
            return col
            
    return None

def identify_team_column(df: pd.DataFrame) -> Optional[str]:
    """identify the column containing team names"""
    # For FBref data, the column is always 'Squad'
    if 'Squad' in df.columns:
        return 'Squad'
    
    # Fallbacks for other formats
    team_column_candidates = ['squad', 'team', 'club']
    
    # Check exact matches
    for col in team_column_candidates:
        if col in df.columns:
            return col
    
    # Case insensitive search
    for col in df.columns:
        if any(c.lower() == col.lower() for c in team_column_candidates):
            return col
            
    return None

def standardize_column_names(df: pd.DataFrame, column_mapping: Dict[str, str]) -> pd.DataFrame:
    """rename columns according to the provided mapping"""
    # create a copy to avoid modifying the original dataframe
    df = df.copy()
    
    # rename only the columns that exist in the dataframe
    existing_cols = {old: new for old, new in column_mapping.items() if old in df.columns}
    df = df.rename(columns=existing_cols)
    
    return df

def filter_outfield_players(df: pd.DataFrame, position_col: str, excluded_positions: List[str]) -> pd.DataFrame:
    """filter out goalkeepers and keep only outfield players"""
    if position_col in df.columns:
        return df[~df[position_col].isin(excluded_positions)]
    return df

def filter_by_minutes(df: pd.DataFrame, minutes_col: str, min_minutes: int) -> pd.DataFrame:
    """filter players with at least the minimum required minutes"""
    if minutes_col in df.columns:
        # Create a copy to avoid modifying the original dataframe
        df = df.copy()
        
        # Convert minutes to numeric, handling commas and other formatting
        df[minutes_col] = pd.to_numeric(
            df[minutes_col].astype(str).str.replace(',', '').str.replace('-', '0'),
            errors='coerce'
        )
        
        # Filter by minimum minutes
        return df[df[minutes_col] >= min_minutes]
    return df

def validate_schema(df: pd.DataFrame, required_columns: List[str]) -> bool:
    """validate that the dataframe contains all required columns"""
    missing_columns = [col for col in required_columns if col not in df.columns]
    if missing_columns:
        print(f"error: missing required columns: {missing_columns}")
        return False
    return True


## 4. Load and Explore Raw Data


In [119]:
# list all csv files in the raw data directory
csv_files = list_csv_files(RAW_DATA_DIR)
print(f"found {len(csv_files)} csv files in {RAW_DATA_DIR}")

# display the file names
for file in csv_files:
    print(f"- {os.path.basename(file)}")


found 9 csv files in ../data/raw/fbref_2024_25/
- fbref_misc_2024_25.csv
- fbref_passing_2024_25.csv
- fbref_playtime_2024_25.csv
- fbref_defense_2024_25.csv
- fbref_standard_2024_25.csv
- fbref_pass_types_2024_25.csv
- fbref_creation_2024_25.csv
- fbref_possession_2024_25.csv
- fbref_shooting_2024_25.csv


In [120]:
# load each csv file into a dictionary of dataframes
raw_data = {}
for file in csv_files:
    table_name = os.path.basename(file).replace('fbref_', '').replace('_2024_25.csv', '')
    try:
        # FBref CSV files have a specific format:
        # - First row contains category headers
        # - Second row contains the actual column names
        # We need to skip the first row and use the second row as headers
        
        # Read the file with the second row as headers
        df = pd.read_csv(file, header=1)
        
        # Clean up the 'Matches' column which often appears at the end of FBref tables
        if 'Matches' in df.columns:
            df = df.drop('Matches', axis=1)
        
        # Check if we have valid data - the first row should have Rk=1
        if 'Rk' in df.columns and len(df) > 0:
            # Ensure we have the player data
            if 'Player' in df.columns and df['Player'].notna().sum() > 0:
                print(f"Data looks good: found {df['Player'].notna().sum()} players with names")
            else:
                print(f"Warning: Player column is missing or empty in {table_name}")
        
        # Add a row number column as a fallback for player identification
        df['row_id'] = range(len(df))
        
        # Store the dataframe
        raw_data[table_name] = df
        print(f"loaded {table_name} data: {df.shape[0]} rows, {df.shape[1]} columns")
    except Exception as e:
        print(f"error loading {table_name} data: {e}")


Data looks good: found 596 players with names
loaded misc data: 596 rows, 25 columns
Data looks good: found 596 players with names
loaded passing data: 596 rows, 32 columns
Data looks good: found 730 players with names
loaded playtime data: 730 rows, 30 columns
Data looks good: found 596 players with names
loaded defense data: 596 rows, 25 columns
Data looks good: found 596 players with names
loaded standard data: 596 rows, 37 columns
Data looks good: found 596 players with names
loaded pass_types data: 596 rows, 24 columns
Data looks good: found 596 players with names
loaded creation data: 596 rows, 25 columns
Data looks good: found 596 players with names
loaded possession data: 596 rows, 31 columns
Data looks good: found 596 players with names
loaded shooting data: 596 rows, 26 columns


In [121]:
# examine the standard table which should have player info
if 'standard' in raw_data:
    print("standard table columns:")
    print(raw_data['standard'].columns.tolist())
    print("\nfirst few rows:")
    display(raw_data['standard'].head())
else:
    print("standard table not found")


standard table columns:
['Rk', 'Player', 'Nation', 'Pos', 'Squad', 'Age', 'Born', 'MP', 'Starts', 'Min', '90s', 'Gls', 'Ast', 'G+A', 'G-PK', 'PK', 'PKatt', 'CrdY', 'CrdR', 'xG', 'npxG', 'xAG', 'npxG+xAG', 'PrgC', 'PrgP', 'PrgR', 'Gls.1', 'Ast.1', 'G+A.1', 'G-PK.1', 'G+A-PK', 'xG.1', 'xAG.1', 'xG+xAG', 'npxG.1', 'npxG+xAG.1', 'row_id']

first few rows:


,Rk,Player,Nation,Pos,Squad,Age,Born,MP,Starts,Min,90s,Gls,Ast,G+A,G-PK,PK,PKatt,CrdY,CrdR,xG,npxG,xAG,npxG+xAG,PrgC,PrgP,PrgR,Gls.1,Ast.1,G+A.1,G-PK.1,G+A-PK,xG.1,xAG.1,xG+xAG,npxG.1,npxG+xAG.1,row_id
0,1,Max Aarons,eng ENG,DF,Bournemouth,24,2000,3,1,86,1,0,0,0,0,0,0,0,0,0,0,0,0,1,8,3,0,0,0,0,0,0,0,0,0,0,0
1,2,Joshua Acheampong,eng ENG,DF,Chelsea,18,2006,4,2,170,1.9,0,0,0,0,0,0,1,0,0.2,0.2,0,0.2,0,8,0,0,0,0,0,0,0.12,0,0.12,0.12,0.12,1
2,3,Tyler Adams,us USA,MF,Bournemouth,25,1999,28,21,"1,965",21.8,0,3,3,0,0,0,7,0,1.6,1.6,1,2.6,14,76,10,0,0.14,0.14,0,0.14,0.07,0.05,0.12,0.07,0.12,2
3,4,Tosin Adarabioyo,eng ENG,DF,Chelsea,26,1997,22,15,"1,409",15.7,1,1,2,1,0,0,4,0,0.9,0.9,0.2,1.2,5,42,1,0.06,0.06,0.13,0.06,0.13,0.06,0.01,0.07,0.06,0.07,3
4,5,Simon Adingra,ci CIV,"FW,MF",Brighton,22,2002,29,12,"1,097",12.2,2,2,4,2,0,0,0,0,2.5,2.5,2.5,4.9,50,18,136,0.16,0.16,0.33,0.16,0.33,0.2,0.2,0.4,0.2,0.4,4


## 5. Clean and Standardize Column Names


In [122]:
# clean column names for all dataframes
cleaned_data = {}
for table_name, df in raw_data.items():
    cleaned_data[table_name] = clean_column_names(df)
    
# display the cleaned column names for the standard table
if 'standard' in cleaned_data:
    print("cleaned standard table columns:")
    print(cleaned_data['standard'].columns.tolist())


cleaned standard table columns:
['rk', 'player', 'nation', 'pos', 'squad', 'age', 'born', 'mp', 'starts', 'min', '90s', 'gls', 'ast', 'gplusa', 'g_pk', 'pk', 'pkatt', 'crdy', 'crdr', 'xg', 'npxg', 'xag', 'npxgplusxag', 'prgc', 'prgp', 'prgr', 'gls1', 'ast1', 'gplusa1', 'g_pk1', 'gplusa_pk', 'xg1', 'xag1', 'xgplusxag', 'npxg1', 'npxgplusxag1', 'row_id']


In [123]:
# identify key columns in each dataframe
key_columns = {}
for table_name, df in cleaned_data.items():
    key_columns[table_name] = {
        'player': identify_player_column(df),
        'minutes': identify_minutes_column(df),
        'position': identify_position_column(df),
        'team': identify_team_column(df)
    }
    print(f"{table_name} key columns: {key_columns[table_name]}")


misc key columns: {'player': 'player', 'minutes': None, 'position': 'pos', 'team': 'squad'}
passing key columns: {'player': 'player', 'minutes': None, 'position': 'pos', 'team': 'squad'}
playtime key columns: {'player': 'player', 'minutes': 'min', 'position': 'pos', 'team': 'squad'}
defense key columns: {'player': 'player', 'minutes': None, 'position': 'pos', 'team': 'squad'}
standard key columns: {'player': 'player', 'minutes': 'min', 'position': 'pos', 'team': 'squad'}
pass_types key columns: {'player': 'player', 'minutes': None, 'position': 'pos', 'team': 'squad'}
creation key columns: {'player': 'player', 'minutes': None, 'position': 'pos', 'team': 'squad'}
possession key columns: {'player': 'player', 'minutes': None, 'position': 'pos', 'team': 'squad'}
shooting key columns: {'player': 'player', 'minutes': None, 'position': 'pos', 'team': 'squad'}


In [124]:
# create a column mapping dictionary to standardize column names across tables
column_mapping = {}
for table_name, columns in key_columns.items():
    for key, value in columns.items():
        if value is not None and value != key:
            column_mapping[value] = key

print("column mapping:")
print(column_mapping)


column mapping:
{'pos': 'position', 'squad': 'team', 'min': 'minutes'}


In [125]:
# standardize column names across all tables
standardized_data = {}
for table_name, df in cleaned_data.items():
    standardized_data[table_name] = standardize_column_names(df, column_mapping)
    
# verify standardization for the standard table
if 'standard' in standardized_data:
    print("standardized standard table columns:")
    print([col for col in standardized_data['standard'].columns if col in REQUIRED_COLUMNS])


standardized standard table columns:
['player', 'position', 'team', 'minutes']


## 6. Filter Players


In [126]:
# start with the standard table as the base for player filtering
if 'standard' in standardized_data:
    base_df = standardized_data['standard'].copy()
    
    # Check if position column has any non-NaN values
    if 'position' in base_df.columns and base_df['position'].notna().sum() > 0:
        # filter outfield players
        base_df = filter_outfield_players(base_df, 'position', EXCLUDED_POSITIONS)
        print(f"filtered outfield players: {base_df.shape[0]} remaining")
        
        # display position distribution
        position_counts = base_df['position'].value_counts()
        print("\nposition distribution:")
        if len(position_counts) > 0:
            print(position_counts)
        else:
            print("No position data available - all values are NaN")
    else:
        print("Position column is empty or missing")
    
    # filter by minimum minutes
    if 'minutes' in base_df.columns:
        base_df = filter_by_minutes(base_df, 'minutes', MIN_MINUTES)
        print(f"filtered by minimum minutes ({MIN_MINUTES}): {base_df.shape[0]} remaining")
else:
    print("standard table not found, cannot filter players")


filtered outfield players: 552 remaining

position distribution:
position
DF       186
MF       112
FW        85
FW,MF     60
MF,FW     44
Pos       22
DF,MF     16
MF,DF     13
FW,DF      7
DF,FW      7
Name: count, dtype: int64
filtered by minimum minutes (600): 340 remaining


## 7. Aggregate Player Data Across Teams


In [127]:
# check if there are players who played for multiple teams
if 'standard' in standardized_data:
    player_col = identify_player_column(standardized_data['standard'])
    team_col = identify_team_column(standardized_data['standard'])
    
    print(f"Identified player column: {player_col}")
    print(f"Identified team column: {team_col}")
    
    if player_col and team_col and player_col in standardized_data['standard'].columns and team_col in standardized_data['standard'].columns:
        # Check if both columns have non-NaN values
        if standardized_data['standard'][player_col].notna().sum() > 0 and standardized_data['standard'][team_col].notna().sum() > 0:
            # Check for duplicate players across teams
            try:
                player_team_counts = standardized_data['standard'].groupby(player_col)[team_col].nunique()
                multi_team_players = player_team_counts[player_team_counts > 1]
                
                print(f"found {len(multi_team_players)} players who played for multiple teams:")
                if len(multi_team_players) > 0:
                    for player in multi_team_players.index:
                        teams = standardized_data['standard'][standardized_data['standard'][player_col] == player][team_col].unique()
                        print(f"- {player}: {', '.join(teams)}")
            except Exception as e:
                print(f"Error checking for multi-team players: {e}")
                print("Continuing with data processing...")
        else:
            print(f"Player or team column contains only NaN values")
    else:
        print(f"Missing player or team column in standard data.")
        print(f"Available columns: {standardized_data['standard'].columns.tolist()}")


Identified player column: player
Identified team column: team
found 12 players who played for multiple teams:
- Axel Disasi: Aston Villa, Chelsea
- Carlos Alcaraz: Southampton, Everton
- Evan Ferguson: West Ham, Brighton
- Jaden Philogene Bidace: Aston Villa, Ipswich Town
- James Ward-Prowse: West Ham, Nott'ham Forest
- Joachim Andersen: Crystal Palace, Fulham
- Jordan Ayew: Crystal Palace, Leicester City
- Julio Enciso: Ipswich Town, Brighton
- Marcus Rashford: Manchester Utd, Aston Villa
- Odsonne Édouard: Crystal Palace, Leicester City
- Reiss Nelson: Arsenal, Fulham
- Trevoh Chalobah: Crystal Palace, Chelsea


In [128]:
# function to aggregate player data across teams
def aggregate_player_data(df: pd.DataFrame, player_col: str = 'player') -> pd.DataFrame:
    """aggregate player statistics across multiple teams"""
    # Make a copy to avoid modifying the original
    df_copy = df.copy()
    
    # Fix for empty player values - if any player values are NaN, we can't aggregate properly
    if player_col in df_copy.columns and df_copy[player_col].isna().any():
        # Fill NaN player values with a placeholder based on row index
        df_copy[player_col] = df_copy[player_col].fillna(df_copy.index.astype(str))
    
    # If player_col is not in the dataframe, use row_id as a fallback
    if player_col not in df_copy.columns and 'row_id' in df_copy.columns:
        player_col = 'row_id'
    
    # If we still don't have a valid player column, return the original dataframe
    if player_col not in df_copy.columns:
        return df_copy
    
    # Check if there are any duplicates to aggregate
    if df_copy[player_col].duplicated().sum() == 0:
        return df_copy
    
    # identify numeric columns for summation
    numeric_cols = df_copy.select_dtypes(include=[np.number]).columns.tolist()
    
    # identify categorical columns to keep the most frequent value
    categorical_cols = [col for col in df_copy.columns if col not in numeric_cols and col != player_col]
    
    # create aggregation dictionary
    agg_dict = {}
    for col in numeric_cols:
        if col != player_col:  # Ensure we don't include player_col in agg_dict
            agg_dict[col] = 'sum'
            
    for col in categorical_cols:
        if col != player_col:  # Ensure we don't include player_col in agg_dict
            agg_dict[col] = lambda x: x.value_counts().index[0] if len(x.value_counts()) > 0 else None
    
    # If no columns to aggregate, return the original dataframe
    if not agg_dict:
        return df_copy
    
    # perform aggregation without resetting index immediately
    result = df_copy.groupby(player_col).agg(agg_dict)
    
    # Reset index to get player_col back as a column
    result = result.reset_index()
    
    return result

# aggregate player data for each table
aggregated_data = {}
for table_name, df in standardized_data.items():
    # Identify the player column for this table
    player_col = identify_player_column(df)
    
    if player_col:
        try:
            aggregated_data[table_name] = aggregate_player_data(df, player_col)
            print(f"aggregated {table_name} data: {df.shape[0]} rows → {aggregated_data[table_name].shape[0]} rows")
        except Exception as e:
            print(f"error aggregating {table_name} data: {e}")
            # Use the original data as fallback
            aggregated_data[table_name] = df
    else:
        print(f"skipping {table_name} data: no player column identified")


aggregated misc data: 596 rows → 563 rows
aggregated passing data: 596 rows → 563 rows
aggregated playtime data: 730 rows → 686 rows
aggregated defense data: 596 rows → 563 rows
aggregated standard data: 596 rows → 563 rows
aggregated pass_types data: 596 rows → 563 rows
aggregated creation data: 596 rows → 563 rows
aggregated possession data: 596 rows → 563 rows
aggregated shooting data: 596 rows → 563 rows


## 8. Merge All Tables


In [129]:
# start with the standard table as the base
if 'standard' in aggregated_data and len(aggregated_data['standard']) > 0:
    merged_df = aggregated_data['standard'].copy()
    
    # Identify the player column in the standard table
    base_player_col = identify_player_column(merged_df)
    
    if base_player_col:
        # merge with other tables
        for table_name, df in aggregated_data.items():
            if table_name != 'standard' and len(df) > 0:
                # Identify the player column in this table
                table_player_col = identify_player_column(df)
                
                if table_player_col:
                    # identify columns to merge (exclude player column to avoid duplicates)
                    merge_cols = [col for col in df.columns if col != table_player_col]
                    
                    # Make sure we have at least one column to merge
                    if len(merge_cols) > 0:
                        try:
                            # merge with the base dataframe
                            merged_df = pd.merge(
                                merged_df,
                                df[[table_player_col] + merge_cols],
                                left_on=base_player_col,
                                right_on=table_player_col,
                                how='left',
                                suffixes=('', f'_{table_name}')
                            )
                            
                            # If the right player column was added with a suffix, drop it
                            if table_player_col != base_player_col and f"{table_player_col}_{table_name}" in merged_df.columns:
                                merged_df = merged_df.drop(f"{table_player_col}_{table_name}", axis=1)
                            
                            print(f"merged with {table_name} data: now {merged_df.shape[1]} columns")
                        except Exception as e:
                            print(f"error merging {table_name} data: {e}")
                else:
                    print(f"skipping {table_name} data: no player column identified")
        
        # check for duplicate column names after merging
        duplicate_cols = merged_df.columns[merged_df.columns.duplicated()].tolist()
        if duplicate_cols:
            print(f"warning: found duplicate column names: {duplicate_cols}")
    else:
        print("No player column found in standard table, cannot merge tables")
else:
    print("Standard table not found or is empty, cannot merge tables")


merged with misc data: now 61 columns
merged with passing data: now 92 columns
merged with playtime data: now 121 columns
merged with defense data: now 145 columns
merged with pass_types data: now 168 columns
merged with creation data: now 192 columns
merged with possession data: now 222 columns
merged with shooting data: now 247 columns


In [130]:
# handle duplicate columns if any
if 'merged_df' in locals() and duplicate_cols:
    # drop duplicate columns
    merged_df = merged_df.loc[:, ~merged_df.columns.duplicated()]
    print(f"dropped duplicate columns: now {merged_df.shape[1]} columns")


In [131]:
# add source table information
if 'merged_df' in locals() and len(merged_df) > 0:
    # create a dictionary to track which columns came from which table
    column_sources = {}
    for table_name, df in standardized_data.items():
        for col in df.columns:
            # handle potential suffixes from merging
            if col in merged_df.columns:
                column_sources[col] = table_name
            elif f"{col}_{table_name}" in merged_df.columns:
                column_sources[f"{col}_{table_name}"] = table_name
    
    # add a source_table column
    merged_df['data_source'] = 'fbref_2024_25'
    
    # Make sure we have the required columns
    for col in REQUIRED_COLUMNS:
        if col not in merged_df.columns:
            # If 'team' is missing but we have 'squad', rename it
            if col == 'team' and 'squad' in merged_df.columns:
                merged_df['team'] = merged_df['squad']
            # If 'squad' is missing but we have 'team', rename it
            elif col == 'squad' and 'team' in merged_df.columns:
                merged_df['squad'] = merged_df['team']
    
    print(f"final dataset: {merged_df.shape[0]} rows, {merged_df.shape[1]} columns")
else:
    print("merged dataframe not created or is empty")


final dataset: 563 rows, 248 columns


## 9. Validate Final Dataset


In [132]:
# validate the schema of the final dataset
if 'merged_df' in locals():
    is_valid = validate_schema(merged_df, REQUIRED_COLUMNS)
    print(f"schema validation: {'passed' if is_valid else 'failed'}")
    
    # check for missing values in key columns
    for col in REQUIRED_COLUMNS:
        if col in merged_df.columns:
            missing_count = merged_df[col].isna().sum()
            print(f"column '{col}': {missing_count} missing values ({missing_count/len(merged_df)*100:.2f}%)")
    
    # display sample of the final dataset
    print("\nsample of the final dataset:")
    display(merged_df[REQUIRED_COLUMNS + ['data_source']].head())
else:
    print("merged dataframe not created, cannot validate schema")


schema validation: passed
column 'player': 0 missing values (0.00%)
column 'team': 0 missing values (0.00%)
column 'position': 0 missing values (0.00%)
column 'minutes': 0 missing values (0.00%)

sample of the final dataset:


,player,team,position,minutes,data_source
0,Aaron Cresswell,West Ham,DF,824,fbref_2024_25
1,Aaron Ramsdale,Southampton,GK,"2,700",fbref_2024_25
2,Aaron Wan-Bissaka,West Ham,DF,"3,154",fbref_2024_25
3,Abdoulaye Doucouré,Everton,MF,"2,564",fbref_2024_25
4,Abdukodir Khusanov,Manchester City,DF,503,fbref_2024_25


## 10. Save Cleaned Dataset


In [133]:
# ensure the output directory exists
os.makedirs(PROCESSED_DATA_DIR, exist_ok=True)

# save the cleaned dataset
if 'merged_df' in locals() and is_valid:
    merged_df.to_csv(OUTPUT_FILE, index=False)
    print(f"saved cleaned dataset to {OUTPUT_FILE}")
    
    # create and save schema information
    schema_info = {
        'num_rows': len(merged_df),
        'num_columns': len(merged_df.columns),
        'columns': {}
    }
    
    for col in merged_df.columns:
        schema_info['columns'][col] = {
            'dtype': str(merged_df[col].dtype),
            'source_table': column_sources.get(col, 'unknown'),
            'missing_values': int(merged_df[col].isna().sum()),
            'unique_values': int(merged_df[col].nunique()) if merged_df[col].dtype != 'object' or merged_df[col].nunique() < 100 else 'too many to list'
        }
    
    with open(SCHEMA_FILE, 'w') as f:
        json.dump(schema_info, f, indent=2)
    
    print(f"saved schema information to {SCHEMA_FILE}")
else:
    print("dataset not saved due to validation failure or missing dataframe")


saved cleaned dataset to ../data/processed/player_stats_cleaned.csv
saved schema information to ../data/processed/data_schema.json


## 11. Summary and Next Steps


In [134]:
# print summary statistics
if 'merged_df' in locals():
    print("Data Cleaning Summary:")
    print(f"- Loaded {len(raw_data)} raw data tables")
    print(f"- Processed {merged_df.shape[0]} unique players")
    print(f"- Final dataset has {merged_df.shape[1]} columns")
    print(f"- Excluded {len(EXCLUDED_POSITIONS)} position types: {EXCLUDED_POSITIONS}")
    print(f"- Minimum minutes threshold: {MIN_MINUTES}")
    
    # print next steps
    print("\nNext Steps (Phase 2):")
    print("- Feature Engineering")
    print("- Per-90 normalization")
    print("- Composite indices calculation (PI, CCI, DA, FE)")
    print("- Winsorization at 5th and 95th percentiles")


Data Cleaning Summary:
- Loaded 9 raw data tables
- Processed 563 unique players
- Final dataset has 248 columns
- Excluded 1 position types: ['GK']
- Minimum minutes threshold: 600

Next Steps (Phase 2):
- Feature Engineering
- Per-90 normalization
- Composite indices calculation (PI, CCI, DA, FE)
- Winsorization at 5th and 95th percentiles
